source: https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/summarization.ipynb#scrollTo=IreSlFmlIrIm

In [1]:
import os

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
)
from datasets import load_dataset
import evaluate
import nltk
import numpy as np
import wandb

nltk.download("punkt", quiet=True)


2023-02-09 03:18:13.285730: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-09 03:18:13.815411: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/torch/lib:/usr/local/lib/python3.8/dist-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-09 03:18:13.815450: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object 

True

In [2]:
ft_output_dir = os.getenv("HF_FINETUNE_OUTPUT_DIR")
checkpoint = "t5-base"
model_name = checkpoint.split("/")[-1]
dataset_name = "xsum"
hub_model_id = f"{model_name}-{dataset_name}"
model_output_dir = os.path.join(ft_output_dir, hub_model_id)

os.environ['WANDB_PROJECT'] = hub_model_id

In [3]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint, low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [4]:
ds = load_dataset(dataset_name)
ds


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [5]:
train_cutoff = 500
ds['train'] = ds['train'].select(range(train_cutoff))
ds['validation'] = ds['validation'].select(range(100))

ds['test'] = ds['test'].select(range(100))

In [6]:
example = ds["train"][0]
print(example["document"], "\n")
print(example["summary"])


The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is al

In [7]:
def preprocess(examples):
    inputs = ["summarize: " + doc for doc in examples["document"]]
    model_input = tokenizer(inputs)
    model_input["labels"] = tokenizer(examples["summary"])["input_ids"]
    return model_input


In [8]:
tk_ds = ds.map(preprocess, batched=True).remove_columns(ds["train"].column_names)
tk_ds

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

In [9]:
rouge = evaluate.load("rouge")


In [10]:
# create baseline with lead_3 summary
lead_3_summary = [
    "\n".join(nltk.sent_tokenize(doc)[:3]) for doc in ds["validation"]["document"]
]
baseline_rouge = rouge.compute(
    predictions=lead_3_summary, references=ds["validation"]["summary"]
)
baseline_rouge


{'rouge1': 0.183906049523487,
 'rouge2': 0.03061047286323782,
 'rougeL': 0.11813915659489206,
 'rougeLsum': 0.14343901685710014}

In [11]:
args = Seq2SeqTrainingArguments(
    output_dir=model_output_dir,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    bf16=True,  # TODO: figure out detail
    gradient_accumulation_steps=8,  # TODO: figure out detail
    predict_with_generate=True,
    report_to="wandb",
    hub_model_id=hub_model_id,
)

In [12]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]

    result = rouge.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    return result


In [13]:
collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    # padding="longest",
    # max_length=512,
    pad_to_multiple_of=8,
)

In [14]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tk_ds["train"],
    eval_dataset=tk_ds["validation"],
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics,
)


Using cuda_amp half precision backend


In [15]:
trainer.train()


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 500
  Num Epochs = 2
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 8
  Total optimization steps = 30
  Number of trainable parameters = 222903552
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: lukaemon. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/trainer.py:1543 in train                     │
│                                                                                                  │
│   1540 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1541 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1542 │   │   )                                                                                 │
│ ❱ 1543 │   │   return inner_training_loop(                                                       │
│   1544 │   │   │   args=args,                                                                    │
│   1545 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1546 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/trainer.py:1791 in _inner_training_loop      │
│                                                                                                  │
│   1788 │   │   │   │   │   with model.no_sync():                                                 │
│   1789 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1790 │   │   │   │   else:                                                                     │
│ ❱ 1791 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1792 │   │   │   │                                                                             │
│   1793 │   │   │   │   if (                                                                      │
│   1794 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/trainer.py:2539 in training_step             │
│                                                                                                  │
│   2536 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2537 │   │                                                                                     │
│   2538 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2539 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2540 │   │                                                                                     │
│   2541 │   │   if self.args.n_gpu > 1:                                                           │
│   2542 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/trainer.py:2571 in compute_loss              │
│                                                                                                  │
│   2568 │   │   │   labels = inputs.pop("labels")                                                 │
│   2569 │   │   else:                                       

In [16]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 100
  Batch size = 4
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│ ❱ 1 trainer.evaluate()                                                                           │
│   2                                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/trainer_seq2seq.py:78 in evaluate            │
│                                                                                                  │
│    75 │   │   )                                                                                  │
│    76 │   │   self._gen_kwargs = gen_kwargs                                                      │
│    77 │   │                                                                                      │
│ ❱  78 │   │   return super().evaluate(eval_dataset, ignore_keys=ignore_keys, metric_key_prefix   │
│    79 │                                                                                          │
│    80 │   def predict(                                                                           │
│    81 │   │   self,                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/trainer.py:2827 in evaluate                  │
│                                                                                                  │
│   2824 │   │   start_time = time.time()                                                          │
│   2825 │   │                                                                                     │
│   2826 │   │   eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else se  │
│ ❱ 2827 │   │   output = eval_loop(                                                               │
│   2828 │   │   │   eval_dataloader,                                                              │
│   2829 │   │   │   description="Evaluation",                                                     │
│   2830 │   │   │   # No point gathering the predictions if there are no metrics, otherwise we d  │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/trainer.py:3009 in evaluation_loop           │
│                                                                                                  │
│   3006 │   │   │   │   │   batch_size = observed_batch_size                                      │
│   3007 │   │   │                                                                                 │
│   3008 │   │   │   # Prediction step                                                             │
│ ❱ 3009 │   │   │   loss, logits, labels = self.prediction_step(model, inputs, prediction_loss_o  │
│   3010 │   │   │   inputs_decode = self._prepare_input(inputs["input_ids"]) if args.include_inp  │
│   3011 │   │   │                                                                                 │
│   3012 │   │   │   if is_torch_tpu_available():                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/trainer_seq2seq.py:198 in prediction_step    │
│                                                                                                  │
│   195 │   │   else:                                                                              │
│   196 │   │   │   generation_inputs = inputs[self.model.mai

In [17]:
wandb.finish()